# Data Analysis and Modeling Instructions

## 1. Load the Data
- Load the dataset and convert it into a GeoDataFrame.
- Ensure the data is projected into the correct local projection.

In [3]:
import esda
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pyinterpolate
from libpysal import graph
from sklearn import ensemble, metrics, model_selection, svm
from sklearn.metrics import mean_absolute_error as mae
import tobler
import statsmodels.formula.api as sm


In [ ]:
url = "https://martinfleischmann.net/sds/tree_regression/data/prices_data.csv"
df_prices = pd.read_csv(url)
df_prices.head()

: 

: 

: 


## 2. Feature Selection
- Choose **at least five independent variables** that you believe will serve as good predictors for the target variable.


In [ ]:
gdf = gpd.GeoDataFrame(df_prices, geometry=gpd.points_from_xy(df_prices.Longitude, df_prices.Latitude), crs="EPSG:4326")
gdf = gdf.drop(columns=["Longitude", "Latitude"])
gdf.head()


: 

: 

: 

In [ ]:
convex_hull = gdf.union_all().convex_hull
gdf_convex = gpd.GeoDataFrame(geometry=[convex_hull], crs="EPSG:4326")
gdf_hexgrid = tobler.util.h3fy(gdf_convex, resolution=9)
gdf_agg_grid = gpd.sjoin(gdf, gdf_hexgrid, how="inner", predicate='within')
gdf_agg_grid = gdf_agg_grid.rename(columns={"hex_id_right": "agg_grid"})




: 

: 

: 

- Let's aggregate all points to hex grid and calculate mean

In [ ]:
def generate_agg_dict(columns):
    agg_dict = {}
    for column in columns:
        agg_dict[column] = "mean"
    return agg_dict

: 

: 

: 

In [ ]:
gdf_agg_grid.columns

: 

: 

: 

In [ ]:
columns = gdf_agg_grid.columns.drop(["geometry", "agg_grid", "hex_id_left"])
agg_dict = generate_agg_dict(columns)

: 

: 

: 

In [ ]:
gdf_agg_grid_agg = gdf_agg_grid.groupby("agg_grid").agg(agg_dict,).reset_index()
gdf_agg_grid_agg = gdf_agg_grid_agg.merge(gdf_hexgrid, left_on="agg_grid", right_on="hex_id", how="inner", validate="one_to_one")

: 

: 

: 

In [ ]:
gdf_agg_grid_agg = gpd.GeoDataFrame(gdf_agg_grid_agg, crs="EPSG:4326")

: 

: 

: 

- plot the data to explore spatial patterns of variables

In [ ]:
variables = ['Property Prices', 'Size', 'Floor', 'Highest floor', 'Units', 'Parking',
            'Heating', 'Year', 'Dist. Green', 'Dist. Water', 'Green Index',
            'Dist. Subway', 'Bus Stop', 'Dist. CBD', 'Top Univ.', 'High School']

fig, axs = plt.subplots(4, 4, figsize=(20, 20))
for variable, ax in zip(variables, axs.flatten()):
    print(variable)
    gdf_agg_grid_agg.plot(
        column=variable,
        ax=ax,
        cmap="magma", 
        legend=True,
        legend_kwds={"shrink": 0.5}
    )
    ax.set_title(variable, fontdict={"fontsize": 8})
    ax.set_axis_off()
    plt.tight_layout()



: 

: 

: 


## 3. Data Preparation
- Split the data into **training** and **testing sets** to prepare for modeling.



In [ ]:
gdf.columns = gdf.columns.str.replace(' ', '_')

gdf.columns = gdf.columns.str.replace('.', '')

: 

: 

: 

In [ ]:
independent_variables_mod1= [
    "Size",
    "Floor",
    "Parking",
    "Heating",
    "Dist_Water",
    "Bus_Stop",
]

independent_variables_mod2 = [
    "Size",
    "Floor",
    "Parking",
    "Dist_Subway",
    "Bus_Stop",
]

independent_variables_mod3 = [
    "Size",
    "Floor",
    "Parking",
    "Units",
    "Heating",
    "Dist_Water",
    "Dist_Subway",   
]

independent1 = gdf[independent_variables_mod1]
independent2 = gdf[independent_variables_mod2]
independent3 = gdf[independent_variables_mod3]


target = gdf["Property_Prices"]

: 

: 

: 

#### Linear regression model

In [ ]:
formula = f"Property_Prices ~ {' + '.join(independent_variables_mod1)}"
formula

: 

: 

: 

In [ ]:
gdf.columns

: 

: 

: 

In [ ]:
gdf_subset = gdf[independent_variables_mod1 + ["Property_Prices"]]
gdf_subset.head()
ols = sm.ols(formula=formula, data=gdf_subset).fit()

: 

: 

: 

In [ ]:
ols.summary()

: 

: 

: 

In [ ]:
r_squared = ols.rsquared

y = gdf_subset["Property_Prices"]
y_pred = ols.predict(gdf_subset[independent_variables_mod1])
mae_val = mae(y, y_pred)

: 

: 

: 

- Regressor 1

In [ ]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(
    independent1, target, test_size=0.2, random_state=0
)
X_train.head()
basic_model = ensemble.RandomForestRegressor(n_jobs=-1, random_state=0)
basic_model.fit(X_train, y_train)
pred_test = basic_model.predict(X_test)
mae1 = metrics.mean_absolute_error(y_test, pred_test)
rsquared1 = metrics.r2_score(y_test, pred_test)

: 

: 

: 

- Regressor 2

In [ ]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(
    independent2, target, test_size=0.2, random_state=0
)
X_train.head()
basic_model = ensemble.RandomForestRegressor(n_jobs=-1, random_state=0)
basic_model.fit(X_train, y_train)
pred_test = basic_model.predict(X_test)
mae2 = metrics.mean_absolute_error(y_test, pred_test)
rsquared2 = metrics.r2_score(y_test, pred_test)

: 

: 

: 

- Regressor 3

In [ ]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(
    independent3, target, test_size=0.2, random_state=0
)
X_train.head()
basic_model = ensemble.RandomForestRegressor(n_jobs=-1, random_state=0)
basic_model.fit(X_train, y_train)
pred_test = basic_model.predict(X_test)
mae3 = metrics.mean_absolute_error(y_test, pred_test)
rsquared3 = metrics.r2_score(y_test, pred_test)

: 

: 

: 

## 4. Model Comparison
- Fit a **linear regression model** and compare its performance with a **tree-based model** of your choice.
- Investigate if there is a noticeable difference between these models.

In [ ]:
print(f"Model 0 - OLS: MAE: {mae_val}, R^2: {r_squared}")
print(f"Model 1: MAE: {mae1}, R^2: {rsquared1}")
print(f"Model 2: MAE: {mae2}, R^2: {rsquared2}")
print(f"Model 3: MAE: {mae3}, R^2: {rsquared3}")


: 

: 

: 

## 5. Model Evaluation
- **Cross-validate** your models to assess their performance.
- **Plot the residuals** to understand the errors.



In [ ]:
gdf.assign(pred_test=pd.Series(pred_test, index=X_test.index))

: 

: 

: 

In [ ]:
pred_cross_val = model_selection.cross_val_predict(
    basic_model,
    independent3,
    target,
    n_jobs=-1,
)
pred_cross_val

: 

: 

: 

In [ ]:
r2_cross_val = metrics.r2_score(
    target, pred_cross_val
)
mae_cross_val = metrics.mean_absolute_error(
    target, pred_cross_val
)
rmse_cross_val = metrics.mean_squared_error(
    target, pred_cross_val, squared=False
)

summary = f"""\
Random Forest (k-fold metrics):
  R2:   {round(r2_cross_val, 3)}
  MAE:  {round(mae_cross_val, 3)}
  RMSE: {round(rmse_cross_val, 3)}
"""
print(summary)

: 

: 

: 

In [ ]:
residuals = (target - pred_cross_val)
residuals.head()

: 

: 

: 

In [ ]:
minmax = residuals.abs().std()
minmax

: 

: 

: 

## 6. Spatial Evaluation
- Perform a **spatial evaluation** using the `"hex_id"` column.
  - Compare **blocked spatial metrics** (based on `"hex_id"`) with **smoothed spatial metrics** by plotting them on a graph.

In [ ]:
gdf["prediction"] = pred_cross_val

: 

: 

: 

In [ ]:
grouped = gdf.groupby("hex_id")[
    ["Property_Prices", "prediction"]
]

block_mae = grouped.apply(
    lambda group: metrics.mean_absolute_error(
        group["Property_Prices"], group["prediction"]
    )
)
block_rmse = grouped.apply(
    lambda group: metrics.root_mean_squared_error(
        group["Property_Prices"], group["prediction"]
    )
)

: 

: 

: 

In [ ]:
spatial_metrics = pd.concat([block_mae, block_rmse], axis=1)
spatial_metrics.columns = ["block_mae", "block_rmse"]
spatial_metrics.head(3)

: 

: 

: 

In [ ]:
gdf_merged = gdf.merge(
    spatial_metrics, left_on="hex_id", right_index=True
)

: 

: 

: 

In [ ]:
fig, axs = plt.subplots(2, 1)
for i, metric in enumerate(["block_mae", "block_rmse"]):
    gdf_merged.plot(metric, ax=axs[i], legend=True, cmap="cividis")
    axs[i].set_title(metric, fontdict={"fontsize": 8})
    axs[i].set_axis_off()

: 

: 

: 

## 7. Variogram Analysis
- Check the spread of the assumed autocorrelation using a **variogram**.

In [ ]:
gdf.geometry

: 

: 

: 

In [ ]:
gdf_projected = gdf.to_crs("EPSG:3857")
input_data = np.hstack(
    [
        gdf_projected.get_coordinates(),
        residuals.abs().values.reshape(-1, 1),
    ]
)

: 

: 

: 

In [ ]:
exp_semivar = pyinterpolate.build_experimental_variogram(
    input_array=input_data,
    step_size=100,
    max_range=10000,
)

: 

: 

: 

In [ ]:
exp_semivar.plot(plot_covariance=False, )

: 

: 

: 

## 8. LISA Analysis
- Conduct **LISA** (Local Indicators of Spatial Association) on the residuals to:
  - Identify **underpriced** and **overpriced properties**.
  - Locate clusters of:
    - Consistently correct predictions.
    - Consistently incorrect predictions.

In [ ]:
distance10km = graph.Graph.build_distance_band(gdf_projected, 10_000)


: 

: 

: 

In [ ]:
moran = esda.Moran_Local(residuals, distance10km)
moran.explore(gdf_projected, tiles="CartoDB Positron")

: 

: 

: 

## 9. Avoiding Spatial Leakage
- Build a new model to avoid **spatial leakage** by using `"hex_id"`.
- **Tune the hyperparameters** of this model.
- Evaluate the new model:
  - Is its performance different from the previous models?
  - Determine the **important features** in this model.

In [ ]:
kf = model_selection.KFold(n_splits=5, shuffle=True)

splits = kf.split(gdf)

split_label = np.empty(len(gdf), dtype=float)
for i, (train, test) in enumerate(splits):
    split_label[test] = i

gdf["split_label"] = split_label

gkf = model_selection.GroupKFold(n_splits=5)

: 

: 

: 

In [ ]:
splits = gkf.split(
    independent1,
    groups=gdf["hex_id"],
)
split_label = np.empty(len(independent1), dtype=float)
for i, (train, test) in enumerate(splits):
    split_label[test] = i

gdf["split_label"] = split_label

gdf.plot(
    "split_label", categorical=True, legend=True, cmap="Set3"
).set_axis_off()

: 

: 

: 

In [ ]:
rf_spatial_cv = ensemble.RandomForestRegressor(random_state=0, n_jobs=-1)

pred_spatial_cv = model_selection.cross_val_predict(
    rf_spatial_cv,
    independent1,
    target,
    groups=gdf["hex_id"],
    cv=gkf,
    n_jobs=-1,
)

: 

: 

: 

In [ ]:
r2_spatial_cv = metrics.r2_score(target, pred_spatial_cv)
mae_spatial_cv = metrics.mean_absolute_error(target, pred_spatial_cv)
rmse_spatial_cv = metrics.root_mean_squared_error(target, pred_spatial_cv)

summary += f"""\
Random Forest with spatial cross-validation (k-fold):
  R2:   {round(r2_spatial_cv, 3)}
  MAE:  {round(mae_spatial_cv, 3)}
  RMSE: {round(rmse_spatial_cv, 3)}
"""
print(summary)

: 

: 

: 

In [ ]:
boosted_tree = ensemble.GradientBoostingRegressor()
pred_boosted_tree = model_selection.cross_val_predict(
    boosted_tree,
    independent1,
    target,
    groups=gdf.hex_id,
    cv=gkf,
)

r2_boosted_tree = metrics.r2_score(target, pred_boosted_tree)
mae_boosted_tree = metrics.mean_absolute_error(target, pred_boosted_tree)
rmse_boosted_tree = metrics.root_mean_squared_error(target, pred_boosted_tree)

summary += f"""\
Gradient Boosted Tree with spatial cross-validation (k-fold):
  R2:   {round(r2_boosted_tree, 3)}
  MAE:  {round(mae_boosted_tree, 3)}
  RMSE: {round(rmse_boosted_tree, 3)}
"""
print(summary)

: 

: 

: 